# Stock Analysis using OpenAI Functions and LangChain Multi Agent

**Diclaimer**: This is only a demo notebook of stocks analysis. The recommendations or advise may not be used for actual invesment decision.

In [1]:
!pip install -q yfinance langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is no

In [2]:
import openai
import json
import os
import numpy as np
import yfinance as yf
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain import LLMMathChain

## Defining a custom method for obtaining stats of a stock

**yfinance** is an open source library to access the financial data available on Yahoo Finance, which offers an excellent range of market data on stocks, bonds and currencies along with market news, reports and analysis.

- Define a function which returns total gains, average and standard deviation of daily prices changes of stocks for a specified number of months

In [3]:
def get_stock_stats(ticker: str, months: int = 1):

    '''
        input string ticker is The correct ticker symbol for the stock in Nation Stock Exchange (NSE) India.
        input integer month is the duration in months for which the statistics of historical prices need to be returned.
    '''

    # Get ticker name correctly
    msft = yf.Ticker(ticker.split(".")[0] + ".NS")

    # get historical market data
    hist = msft.history(period = str(months) + "mo" )

    # Compute the market data
    hist['daily_changes']  = (hist['Close'] - hist['Open']) * 100 / hist['Open']

    # Compute different statistics
    total_gain = (hist.iloc[-1]["Close"] - hist.iloc[0]["Open"] ) * 100 / hist.iloc[0]["Open"]
    avg_daily_changes = np.mean(hist['daily_changes'])
    std_daily_changes = np.std(hist['daily_changes'])

    stock_stats = {'total_gain_in_percentage': round(total_gain, 3),
                   'average_daily_changes_in_percentage': round(avg_daily_changes, 3),
                   'std_daily_changes_in_percentage': round(std_daily_changes, 3)}

    return json.dumps(stock_stats)

## Configuring the function as a tool

## Configuring tool as OpenAI function

In [4]:
from getpass import getpass

In [5]:
os.environ["OPENAI_API_KEY"] = getpass("OpenAI Key:")
openai.api_key  = os.getenv('OPENAI_API_KEY')

OpenAI Key:··········


In [6]:
stockstats = Tool(
    name="stockstats",
    func=get_stock_stats,
    description="get total gains, average and standard deviation of daily prices changes of \
    stocks for a specified number of months"
)

In [7]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [9]:
tools = [
    stockstats
]

agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_MULTI_FUNCTIONS,
                        verbose=True)

In [10]:
import langchain
langchain.debug = True

In [ ]:
agent.run(
    "How to calculate risk of stock?"
)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How to calculate risk of stock?"
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI assistant.\nHuman: How to calculate risk of stock?"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [7.07s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Calculating the risk of a stock involves analyzing various factors such as volatility, beta, standard deviation, and historical performance. Here are some common methods to calculate the risk of a stock:\n\n1. Volatility: Volatility measures the degree of variation in the price of a stock. It is often calculated using the standard deviation of the stock's returns over a specific period. Higher volatility indicates higher risk.\n\n2. Beta: Beta measures the sensitivity of a stock's price movement to the overall market. A beta of 1 indicates t

"Calculating the risk of a stock involves analyzing various factors such as volatility, beta, standard deviation, and historical performance. Here are some common methods to calculate the risk of a stock:\n\n1. Volatility: Volatility measures the degree of variation in the price of a stock. It is often calculated using the standard deviation of the stock's returns over a specific period. Higher volatility indicates higher risk.\n\n2. Beta: Beta measures the sensitivity of a stock's price movement to the overall market. A beta of 1 indicates that the stock moves in line with the market, while a beta greater than 1 indicates higher volatility compared to the market. A beta less than 1 indicates lower volatility.\n\n3. Standard Deviation: Standard deviation measures the dispersion of a stock's returns from its average return. It provides an indication of the stock's volatility and risk. Higher standard deviation implies higher risk.\n\n4. Historical Performance: Analyzing the historical p

In [ ]:
agent.run(
    "What is the return from tata power stock in NSE in last one month?"
)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the return from tata power stock in NSE in last one month?"
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI assistant.\nHuman: What is the return from tata power stock in NSE in last one month?"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [1.99s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call": {
                "name": "tool_selection",
                "arguments": "{\n

'The return from Tata Power stock in NSE in the last one month is -3.994%. The average daily changes in percentage is -0.352% and the standard deviation of daily changes in percentage is 1.041%.'

In [ ]:
#import langchain
#langchain.debug = False

## OpenAI Multi Function Agent

In [ ]:
response = agent.run(
    "Between tata power and adani power in NSE India, which stock had \
    observerd higher volatility in last one month? Explain the steps for your response. \
    The correct ticket symbol for NSE should be generated from the company names and passed as parameter to tools as necessary."
)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Between tata power and adani power in NSE India, which stock had     observerd higher volatility in last one month? Explain the steps for your response."
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI assistant.\nHuman: Between tata power and adani power in NSE India, which stock had     observerd higher volatility in last one month? Explain the steps for your response."
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [3.23s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": 

In [ ]:
print(response)

Based on the data obtained from the stockstats tool, the stock of Tata Power (TATAPOWER.NS) observed higher volatility in the last one month compared to Adani Power (ADANIPOWER.NS).

Here are the steps for determining the response:

1. Retrieve the stock data for Tata Power (TATAPOWER.NS) using the stockstats tool.
2. Calculate the total gain in percentage for Tata Power, which is -3.994%.
3. Calculate the average daily changes in percentage for Tata Power, which is -0.352%.
4. Calculate the standard deviation of daily changes in percentage for Tata Power, which is 1.041%.

5. Retrieve the stock data for Adani Power (ADANIPOWER.NS) using the stockstats tool.
6. Calculate the total gain in percentage for Adani Power, which is -13.478%.
7. Calculate the average daily changes in percentage for Adani Power, which is -0.646%.
8. Calculate the standard deviation of daily changes in percentage for Adani Power, which is 1.706%.

9. Compare the standard deviation of daily changes in percentage 

In [18]:
response = agent.run(
    "Between maruti and adani power in NSE India, which stock had \
    observerd higher volatility in last one month? Explain the steps for your response. \
    The ticker is the NSE symbol for the company and is a single word."

)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Between maruti and adani power in NSE India, which stock had     observerd higher volatility in last one month? Explain the steps for your response.     The ticker is the NSE symbol for the company and is a single word."
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI assistant.\nHuman: Between maruti and adani power in NSE India, which stock had     observerd higher volatility in last one month? Explain the steps for your response.     The ticker is the NSE symbol for the company and is a single word."
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [2.74s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "

ERROR:yfinance:ADANI POWER.NS: No data found, symbol may be delisted


[tool/error] [1:chain:AgentExecutor > 4:tool:stockstats] s] Tool run errored with error:
IndexError('single positional indexer is out-of-bounds')
[chain/error] [1:chain:AgentExecutor] [3.23s] Chain run errored with error:
"IndexError('single positional indexer is out-of-bounds')"


IndexError: ignored